In [1]:
!pip install galois
!pip install Pillow
!pip install scipy

In [2]:
import galois
import numpy as np
from PIL import Image
import random
import math
from scipy.stats import norm

In [3]:
F = galois.GF(2)
n = 7
r = 3 # количество проверочных битов (длина синдрома)

In [4]:
def syndrom(x, n):    
    s = int(0)
    for i in range(1, n+1):  
        if x[i-1] == F(1):  #Если бит x[i-1] равен 1, выполняет XOR синдрома с номером позиции i
            s = s ^ i
    return s
#    1  2  3  4  5  6  7 позиции
x = [0, 1, 0, 0, 0, 0, 0]
print(syndrom(x,7))         # XOR  0 и 2 = 000 xor 010


#    1  2  3  4  5  6  7 позиции
x = [0, 1, 1, 0, 0, 0, 1]
print(syndrom(x,7))      # XOR  2 и 7 = 010 xor   111


2
6


In [5]:
def binvect_to_num(v):
    bits = [int(b) for b in v]  # Преобразуем элементы GF(2) в int
    num = int("".join(str(b) for b in bits), 2)
    return num

print(binvect_to_num([100]))
print(binvect_to_num([101]))


4
5


In [6]:
# Декодирование сообщения
def encode(x_, n, r):
    s_ = syndrom(x_, n)
    bin_str = bin(s_)[2:].zfill(r)
    bits = [int(b) for b in bin_str]
    return F(bits)


In [7]:
# Основная функция встраивания
def task1(x, m, n):
    pos = syndrom(x, n)          # получаем синдром
    message = binvect_to_num(m)  # переводим наше сообщение в 10 вид
    
    if pos == message:          # если syndrom == m, то возвращаем исходный x
        return x
    elif pos != 0:              # если syndrom != 0, то генерируем вектор ошибки e[syndrom + m] = 1
        e = F.Zeros(n)
        error_pos = (message ^ pos) - 1
        e[error_pos] = 1
        return x + e
    else:                       # если syndrom == 0, то генерируем вектор ошибки e[m] = 1  
        e = F.Zeros(n)
        e[message-1] = 1
        return x + e

In [42]:
print("=== Тестирование основных функций ===")
x = F.Random(n)
m = F.Random(r)
print(f"контейнер: {x}")
print(f"сообщение: {m}")

x_ = task1(x, m, n)
m_ = encode(x_, n, r)
print(f"изменённый контейнер: {x_}")
print(f"извлечённое сообщение: {m_}")
print(f"Сообщения совпадают: {np.array_equal(m, m_)}")

=== Тестирование основных функций ===
контейнер: [1 0 1 1 1 0 0]
сообщение: [0 0 1]
изменённый контейнер: [1 1 1 1 1 0 0]
извлечённое сообщение: [0 0 1]
Сообщения совпадают: True


 2 задание

In [10]:
def get_lsb_bits(img):
    """Извлечение младших битов из всех пикселей"""
    pixels = list(img.getdata())
    bits = []
    for p in pixels:
        if isinstance(p, int): # если grayscale
            bits.append(p & 1)
        else:   # для rgb последний бит из tuple                
            for c in p:
                bits.append(c & 1)
    return bits


In [11]:
def set_lsb_bits(img, bits):
    """Замена всех младших битов пикселей на новые после кодирования"""
    pixels = list(img.getdata())
    new_pixels = []
    bit_idx = 0
    total_bits_needed = len(pixels) * (3 if isinstance(pixels[0], tuple) else 1)

    # Если bits короче, дополняем нулями
    if len(bits) < total_bits_needed:
        bits = bits + [0] * (total_bits_needed - len(bits))
    
    for p in pixels:
        if isinstance(p, int):   # если grayscale
            if bit_idx < len(bits):
                val = (p & ~1) | bits[bit_idx]
                new_pixels.append(val)
                bit_idx += 1
            else:
                new_pixels.append(p)  # оставляем исходное значение
        else:  # для rgb
            new_channels = []
            for c in p:
                if bit_idx < len(bits):
                    val = (c & ~1) | bits[bit_idx]
                    new_channels.append(val)
                    bit_idx += 1
                else:
                    new_channels.append(c)  # оставляем исходное значение
            new_pixels.append(tuple(new_channels))

    # Создаем новое изображение с измененными пикселями
    if img.mode == 'RGB':
        new_img = Image.new('RGB', img.size)
    else:
        new_img = Image.new('L', img.size)
    new_img.putdata(new_pixels)
    return new_img

In [12]:
def encode_image_lsb(img_path, message_bits, n, r, output_path):
    img = Image.open(img_path).convert('RGB')
    lsb_bits = get_lsb_bits(img)

    # Разбиваем LSB на блоки длиной n
    blocks_x = [F(lsb_bits[i:i+n]) for i in range(0, len(lsb_bits), n) if i + n <= len(lsb_bits)]
    
    # Разбиваем сообщение на блоки длиной r
    blocks_m = [F(message_bits[i:i+r]) for i in range(0, len(message_bits), r) if i + r <= len(message_bits)]
    
    num_blocks = len(blocks_m)  # количество блоков сообщения

    # Вставляем число блоков в первый блок как заголовок
    header_bits = [int(b) for b in format(num_blocks, f'0{r}b')]
    header_block = F(header_bits)
    blocks_m = [header_block] + blocks_m 

    # Проверка длины
    if len(blocks_m) > len(blocks_x):
        raise ValueError("Сообщение слишком длинное для данного изображения")

    # Кодирование
    encoded_blocks = []
    for i, m_block in enumerate(blocks_m):
        x = blocks_x[i]
        encoded = task1(x, m_block, n)
        encoded_blocks.append(encoded)

    # Оставшиеся блоки LSB (если есть) просто добавляем без изменений
    encoded_blocks += blocks_x[len(blocks_m):]

    # Собираем все биты обратно
    new_bits = []
    for block in encoded_blocks:
        for b in block:
            new_bits.append(int(b))

    # добавляем биты, которые не вошли в полные блоки
    total_processed_bits = len(blocks_x) * n
    if total_processed_bits < len(lsb_bits):
        # Добавляем оставшиеся биты из исходного LSB
        new_bits.extend(lsb_bits[total_processed_bits:])

    new_bits = new_bits[:len(lsb_bits)]

    # Заменяем младшие биты в изображении
    new_img = set_lsb_bits(img, new_bits)
    new_img.save(output_path)
    print(f"Изображение успешно сохранено в {output_path}")

In [13]:
# Декодирование сообщения из изображения
def decode_image_lsb(img_path, n, r):
    img = Image.open(img_path).convert('RGB')
    lsb_bits = get_lsb_bits(img)
    
    # Разбиваем LSB на блоки длиной n
    blocks_x = [F(lsb_bits[i:i+n]) for i in range(0, len(lsb_bits), n) if i + n <= len(lsb_bits)]
    
    # Сначала считываем заголовок -- кол-во блоков сообщения
    header_block = blocks_x[0]
    num_blocks = syndrom(header_block, n) 

    message_bits = []

    # Считываем блоки и декодируем
    for i in range(1, num_blocks + 1):
        if i < len(blocks_x):
            x = blocks_x[i]
            bits_block = encode(x, n, r) 
            message_bits.extend([int(b) for b in bits_block])

    return message_bits

In [14]:
message_bits = [1,0,1,  0,1,1,  1,1,0,  0,0,1,  1,0,0,  1,0,1]  

encode_image_lsb("test1.png", message_bits, n, r, "output.png")
res = decode_image_lsb("output.png",n,r)
print(res)

Изображение успешно сохранено в output.png
[1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1]


In [15]:
message_bits == res

True

# бонус

In [17]:
def generate_pixel_order(img, seed):
    """Возвращает псевдослучайный список индексов пикселей"""
    pixels = list(img.getdata())
    total_bits = len(pixels) * (3 if isinstance(pixels[0], tuple) else 1)
    indices = list(range(total_bits))
    random.seed(seed)
    random.shuffle(indices)
    return indices

In [18]:
def encode_image_lsb_random(img_path, message_bits, n, r, output_path, seed):
    """Кодирование со случайным порядком пикселей"""
    img = Image.open(img_path).convert('RGB')
    lsb_bits = get_lsb_bits(img)
    total_bits = len(lsb_bits)

    # Перемешиваем порядок битов по seed
    pixel_order = generate_pixel_order(img, seed)

    # Разбиваем LSB и сообщение на блоки в новом порядке
    blocks_x = []
    for i in range(0, total_bits - n + 1, n):
        block_bits = [lsb_bits[pixel_order[i + j]] for j in range(n)]
        blocks_x.append(F(block_bits))
    
    blocks_m = [F(message_bits[i:i + r]) for i in range(0, len(message_bits), r) if i + r <= len(message_bits)]

    num_blocks = len(blocks_m)
    header_bits = [int(b) for b in format(num_blocks, f'0{r}b')]
    header_block = F(header_bits)
    blocks_m = [header_block] + blocks_m

    if len(blocks_m) > len(blocks_x):
        raise ValueError("Сообщение слишком длинное для данного изображения")

    encoded_blocks = []
    for i, m in enumerate(blocks_m):
        x = blocks_x[i]
        encoded = task1(x, m, n)
        encoded_blocks.append(encoded)

    # Остальные блоки без изменений
    encoded_blocks += blocks_x[len(blocks_m):]
    
    # Собираем новые биты
    new_bits = []
    for block in encoded_blocks:
        for b in block:
            new_bits.append(int(b))
    new_bits = new_bits[:total_bits]

    # Восстанавливаем оригинальный порядок
    modified_bits = lsb_bits.copy()
    for i, bit in enumerate(new_bits):
        modified_bits[pixel_order[i]] = bit

    new_img = set_lsb_bits(img, modified_bits)
    new_img.save(output_path)
    print(f"Изображение успешно сохранено. Ключ (seed): {seed}")

In [19]:
def decode_image_lsb_random(img_path, n, r, seed):
    """Декодирование со случайным порядком пикселей"""
    img = Image.open(img_path).convert('RGB')
    lsb_bits = get_lsb_bits(img)
    total_bits = len(lsb_bits)

    pixel_order = generate_pixel_order(img, seed)
    
    blocks_x = []
    for i in range(0, total_bits - n + 1, n):
        block_bits = [lsb_bits[pixel_order[i + j]] for j in range(n)]
        blocks_x.append(F(block_bits))

    # Считываем количество блоков
    header_block = blocks_x[0]
    num_blocks = syndrom(header_block, n)

    message_bits = []
    for i in range(1, num_blocks + 1):
        if i < len(blocks_x):
            x = blocks_x[i]
            bits_block = encode(x, n, r)
            message_bits.extend([int(b) for b in bits_block])

    return message_bits

In [20]:
seed = random.randint(0, 2**32 - 1)   # генерация ключа

message_bits = [1,0,1, 1,0,1, 0,0,1, 0,1,1, 0,0,0, 1,1,0]
encode_image_lsb_random("test1.png", message_bits, n=7, r=3,
                        output_path="encoded.png", seed=seed)

decoded = decode_image_lsb_random("encoded.png", n=7, r=3, seed=seed)
print(f"Декодированное сообщение: {decoded}")

Изображение успешно сохранено. Ключ (seed): 2380160004
Декодированное сообщение: [1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0]


In [21]:
decoded == message_bits

True

# 3 задание

In [34]:
import math
from collections import Counter
from PIL import Image

try:
    from scipy.stats import chi2
    from scipy.special import erfc
    SCIPY_AVAILABLE = True
except ImportError:
    SCIPY_AVAILABLE = False
    def erfc(x):
        """Приближенная реализация дополнительной функции ошибок"""
        return 2.0 / (1 + math.exp(2 * x * (1 + 0.14 * x * x)))

class StatisticalTests:
    """Класс для статистического анализа (упрощённые версии тестов NIST)"""

    @staticmethod
    def frequency_test(bits):
        n = len(bits)
        if n == 0:
            return 0.0

        ones = sum(bits)
        zeros = n - ones
        s = ones - zeros  
        s_obs = abs(s) / math.sqrt(n)
        if SCIPY_AVAILABLE:
            p_value = erfc(s_obs / math.sqrt(2))
        else:
            p_value = erfc(s_obs / math.sqrt(2))
        return p_value

    @staticmethod
    def runs_test(bits):
        n = len(bits)
        if n == 0:
            return 0.0
        
        ones = sum(bits)
        pi = ones / n
        
        # Проверка предварительного условия
        if abs(pi - 0.5) >= (2 / math.sqrt(n)):
            return 0.0
        
        # Подсчет серий 
        runs = 1
        for i in range(1, n):
            if bits[i] != bits[i - 1]:
                runs += 1
                
        # Вычисление статистики
        expected_runs = 2 * n * pi * (1 - pi)
        variance = 2 * n * pi * (1 - pi) * (1 - 2 * pi * (1 - pi))
        if variance <= 0:
            return 0.0
            
        z = (runs - expected_runs) / math.sqrt(variance)
        
        if SCIPY_AVAILABLE:
            p_value = 2 * (1 - 0.5 * (1 + math.erf(abs(z) / math.sqrt(2))))
        else:
            p_value = erfc(abs(z) / math.sqrt(2))
        return p_value

    @staticmethod
    def block_frequency_test(bits, M=128):
        """Возвращает p-value и chi-square статистику"""
        n = len(bits)
        if n < M or M == 0:
            return 0.0, 0.0
            
        N = n // M  # Количество блоков
        chi_square = 0.0
        
        for i in range(N):
            block = bits[i*M:(i+1)*M]
            pi = sum(block) / M
            chi_square += (pi - 0.5) ** 2
            
        chi_square *= 4 * M

        if SCIPY_AVAILABLE:
            p_value = chi2.sf(chi_square, N)  
        else:
            # Приближение для p-value когда scipy недоступен
            p_value = math.exp(-chi_square / 2)
            
        return p_value, chi_square

    @staticmethod
    def interpret_p_value(p_value, test_name=""):
        """Интерпретация p-value для изображений"""
        if p_value >= 0.1:
            return " СЛУЧАЙНО (p ≥ 0.1)"
        elif p_value >= 0.01:
            return " ПРИЕМЛЕМО (0.01 ≤ p < 0.1)"
        elif p_value >= 0.001:
            return " СЛАБОЕ ОТКЛОНЕНИЕ (0.001 ≤ p < 0.01)"
        else:
            return " СИЛЬНОЕ ОТКЛОНЕНИЕ (p < 0.001)"
    
    @staticmethod
    def run_all_tests(bits, test_name="", M=128):
        n = len(bits)
        if n == 0:
            print(f"Нет данных для тестирования: {test_name}")
            return
            
        ones = sum(bits)
        proportion = ones / n
        
        print(f"\n{'='*60}")
        print(f"СТАТИСТИЧЕСКИЕ ТЕСТЫ: {test_name}")
        print(f"{'='*60}")
        print(f"Общее количество битов: {n:,}")
        print(f"Доля единиц: {proportion:.6f} (идеально 0.5)")
        print(f"Доля нулей: {1-proportion:.6f}")
        
        # Frequency Test
        p_freq = StatisticalTests.frequency_test(bits)
        freq_interpretation = StatisticalTests.interpret_p_value(p_freq, "Frequency Test")
        print(f"\n1. Frequency (Monobit) Test:")
        print(f"   p-value = {p_freq:.6f}")
        print(f"   Интерпретация: {freq_interpretation}")
        
        # Runs Test
        p_runs = StatisticalTests.runs_test(bits)
        runs_interpretation = StatisticalTests.interpret_p_value(p_runs, "Runs Test")
        print(f"\n2. Runs Test:")
        print(f"   p-value = {p_runs:.6f}")
        print(f"   Интерпретация: {runs_interpretation}")
        
        # Block Frequency Test
        p_block, chi_sq = StatisticalTests.block_frequency_test(bits, M)
        block_interpretation = StatisticalTests.interpret_p_value(p_block, "Block Frequency Test")
        approx_note = " (приближенно)" if not SCIPY_AVAILABLE else ""
        print(f"\n3. Block Frequency Test (M={M}):")
        print(f"   p-value = {p_block:.6f}{approx_note}")
        print(f"   Chi-square = {chi_sq:.3f}")
        print(f"   Интерпретация: {block_interpretation}")
        
        # Сводка
        print(f"\n{'─'*40}")
        print(" ВАЖНО: Для реальных изображений LSB часто НЕ случайны!")
        print("Это нормально и не обязательно указывает на стеганографию.")
        print("Главное — сравнить распределение до и после встраивания.")
        
        return {
            'frequency_p': p_freq,
            'runs_p': p_runs, 
            'block_p': p_block,
            'chi_square': chi_sq,
            'proportion': proportion
        }


def get_lsb_bits(img):
    """Извлечение младших битов из изображения."""
    pixels = list(img.getdata())
    bits = []
    
    for pixel in pixels:
        if isinstance(pixel, int):
            # Grayscale изображение
            bits.append(pixel & 1)
        else:
            # RGB изображение
            for channel in pixel:
                bits.append(channel & 1)
                
    return bits


def analyze_image_lsb_distribution(img_path, test_name="", n_preview_pixels=5):
    """
    Полный анализ распределения LSB в изображении.
    """
    try:
        img = Image.open(img_path).convert('RGB')
    except Exception as e:
        print(f"Ошибка загрузки изображения {img_path}: {e}")
        return None
        
    print(f"\n{'#'*70}")
    print(f"АНАЛИЗ LSB РАСПРЕДЕЛЕНИЯ: {test_name}")
    print(f"Файл: {img_path}")
    print(f"{'#'*70}")
    
    # Базовая информация об изображении
    pixels = list(img.getdata())
    print(f"Размер: {img.size} | Пикселей: {len(pixels):,} | Режим: {img.mode}")
    
    # Пример первых пикселей
    print(f"\nПример первых {n_preview_pixels} пикселей (RGB):")
    for i, pixel in enumerate(pixels[:n_preview_pixels]):
        lsb_bits = [channel & 1 for channel in pixel]
        print(f"  Пиксель {i}: RGB{pixel} → LSB{lsb_bits}")
    
    # Извлекаем LSB биты
    bits = get_lsb_bits(img)
    total_bits = len(bits)
    ones = sum(bits)
    proportion = ones / total_bits
    
    print(f"\n ОБЩАЯ СТАТИСТИКА LSB:")
    print(f"  Всего LSB битов: {total_bits:,}")
    print(f"  Единиц: {ones:,} ({proportion:.4%})")
    print(f"  Нулей: {total_bits-ones:,} ({1-proportion:.4%})")
    
    # Анализ по каналам (для RGB)
    if pixels and isinstance(pixels[0], tuple):
        channels = len(pixels[0])
        print(f"\n СТАТИСТИКА ПО КАНАЛАМ:")
        for channel_idx in range(channels):
            channel_bits = [pixel[channel_idx] & 1 for pixel in pixels]
            channel_ones = sum(channel_bits)
            channel_prop = channel_ones / len(channel_bits)
            channel_name = ['Красный', 'Зелёный', 'Синий'][channel_idx] if channels == 3 else f'Канал {channel_idx}'
            print(f"  {channel_name}: {channel_ones:,} единиц ({channel_prop:.4%})")
    
    # Гистограмма первых битов
    preview_bits = 1000
    if len(bits) >= preview_bits:
        cnt = Counter(bits[:preview_bits])
        print(f"\n ГИСТОГРАММА ПЕРВЫХ {preview_bits} LSB БИТОВ:")
        print(f"  0: {cnt[0]:,} | 1: {cnt[1]:,}")
        print(f"  Соотношение: {cnt[1]/preview_bits:.2%} единиц, {cnt[0]/preview_bits:.2%} нулей")
    
    return bits


def compare_distributions(results):
    """Сравнение распределений между изображениями"""
    print(f"\n{'#'*70}")
    print(" СРАВНЕНИЕ РАСПРЕДЕЛЕНИЙ LSB")
    print(f"{'#'*70}")
    
    print(f"\n{'Изображение':<35} {'Доля единиц':<15} {'Изменение':<15}")
    print(f"{'-'*70}")
    
    # Находим оригинальное изображение
    original_key = None
    for key in results.keys():
        if 'оригинал' in key.lower() or 'original' in key.lower():
            original_key = key
            break
    
    if original_key:
        original_prop = results[original_key]['proportion']
        print(f"{original_key:<35} {original_prop:.6f} {'(эталон)':<15}")
        
        for key, result in results.items():
            if key != original_key:
                prop = result['proportion']
                diff = prop - original_prop
                diff_percent = (diff / original_prop * 100) if original_prop > 0 else 0
                diff_str = f"{diff:+.6f} ({diff_percent:+.2f}%)"
                print(f"{key:<35} {prop:.6f} {diff_str:<15}")
    else:
        for key, result in results.items():
            prop = result['proportion']
            print(f"{key:<35} {prop:.6f}")


if __name__ == "__main__":
    print(" СТАТИСТИЧЕСКИЙ АНАЛИЗ LSB РАСПРЕДЕЛЕНИЯ")
    print("=" * 70)
    print("  Для реальных изображений LSB часто не случайны.")
    print("   Главное — сравнить распределения до и после встраивания.")
    print("=" * 70)
    
    test_files = [
        ("test1.png", "Оригинальное изображение"),
        ("output.png", "Стего-изображение (последовательное)"), 
        ("encoded.png", "Стего-изображение (случайное)")
    ]
    
    results = {}
    
    for file_path, description in test_files:
        try:
            # Анализ распределения LSB
            bits = analyze_image_lsb_distribution(file_path, description, n_preview_pixels=3)
            
            if bits:
                # Статистические тесты на всех битах
                print(f"\n ЗАПУСК СТАТИСТИЧЕСКИХ ТЕСТОВ ДЛЯ: {description}")
                test_result = StatisticalTests.run_all_tests(bits, description, M=128)
                results[description] = test_result
                
        except Exception as e:
            print(f"Ошибка при анализе {file_path}: {e}")
            continue
    
    # Сводная таблица с интерпретацией
    print(f"\n{'#'*70}")
    print(" СВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ")
    print(f"{'#'*70}")
    print(" Условные обозначения:")
    print("    p ≥ 0.1 - Случайное распределение")
    print("     0.01 ≤ p < 0.1 - Приемлемое отклонение")
    print("     0.001 ≤ p < 0.01 - Слабое отклонение")
    print("    p < 0.001 - Сильное отклонение")
    print(f"{'-'*70}")
    
    print(f"\n{'Тест':<35} {'Frequency':<25} {'Runs':<25} {'Block':<25}")
    print(f"{'-'*110}")
    
    for test_name, result in results.items():
        freq_interpret = StatisticalTests.interpret_p_value(result['frequency_p'])
        runs_interpret = StatisticalTests.interpret_p_value(result['runs_p'])
        block_interpret = StatisticalTests.interpret_p_value(result['block_p'])
        
        print(f"{test_name:<35} {freq_interpret:<25} {runs_interpret:<25} {block_interpret:<25}")
    
    # Сравнение распределений
    compare_distributions(results)
    
    print(f"\n{'#'*70}")
    print(" ВЫВОДЫ:")
    print(f"{'#'*70}")
    print("1. Для изображений нормально иметь неслучайное распределение LSB.")
    print("2. Главный показатель успешного встраивания — МИНИМАЛЬНОЕ ИЗМЕНЕНИЕ")
    print("   в распределении LSB после встраивания.")
    print("3. Если распределение почти не изменилось, значит встраивание")
    print("   было проведено корректно и незаметно.")

 СТАТИСТИЧЕСКИЙ АНАЛИЗ LSB РАСПРЕДЕЛЕНИЯ
  Для реальных изображений LSB часто не случайны.
   Главное — сравнить распределения до и после встраивания.

######################################################################
АНАЛИЗ LSB РАСПРЕДЕЛЕНИЯ: Оригинальное изображение
Файл: test1.png
######################################################################
Размер: (1680, 1050) | Пикселей: 1,764,000 | Режим: RGB

Пример первых 3 пикселей (RGB):
  Пиксель 0: RGB(53, 30, 40) → LSB[1, 0, 0]
  Пиксель 1: RGB(53, 30, 40) → LSB[1, 0, 0]
  Пиксель 2: RGB(53, 30, 40) → LSB[1, 0, 0]

 ОБЩАЯ СТАТИСТИКА LSB:
  Всего LSB битов: 5,292,000
  Единиц: 2,969,205 (56.1074%)
  Нулей: 2,322,795 (43.8926%)

 СТАТИСТИКА ПО КАНАЛАМ:
  Красный: 618,095 единиц (35.0394%)
  Зелёный: 1,180,481 единиц (66.9207%)
  Синий: 1,170,629 единиц (66.3622%)

 ГИСТОГРАММА ПЕРВЫХ 1000 LSB БИТОВ:
  0: 93 | 1: 907
  Соотношение: 90.70% единиц, 9.30% нулей

 ЗАПУСК СТАТИСТИЧЕСКИХ ТЕСТОВ ДЛЯ: Оригинальное изображение

СТАТИСТИ

In [38]:
def compare_stego_changes(original_path, stego_path, description, message_bits=18):
    """Сравнение изменений между оригиналом и стегоизображением"""
    try:
        original_img = Image.open(original_path).convert('RGB')
        stego_img = Image.open(stego_path).convert('RGB')
    except Exception as e:
        print(f" Ошибка загрузки изображений: {e}")
        return
    
    print(f"\n{''*35}")
    print(f"СРАВНЕНИЕ: {description}")
    print(f"{''*35}")
    
    # Проверяем размеры
    if original_img.size != stego_img.size:
        print("  Внимание: изображения имеют разный размер!")
        return
    
    # Получаем биты LSB
    original_bits = get_lsb_bits(original_img)
    stego_bits = get_lsb_bits(stego_img)
    
    if len(original_bits) != len(stego_bits):
        print("  Внимание: разное количество LSB битов!")
        return
    
    total_bits = len(original_bits)
    
    # 1. Подсчет измененных битов
    changed_bits = sum(1 for i in range(total_bits) 
                      if original_bits[i] != stego_bits[i])
    unchanged_bits = total_bits - changed_bits
    change_percentage = (changed_bits / total_bits) * 100
    
    # 2. Подсчет изменений по каналам (для RGB)
    pixels = list(original_img.getdata())
    stego_pixels = list(stego_img.getdata())
    
    channel_changes = [0, 0, 0]  # R, G, B
    pixels_with_changes = 0
    pixels_with_single_change = 0
    pixels_with_multiple_changes = 0
    
    for i in range(len(pixels)):
        orig_pixel = pixels[i]
        stego_pixel = stego_pixels[i]
        pixel_changes = 0
        
        for ch in range(3):  # 0=R, 1=G, 2=B
            if (orig_pixel[ch] & 1) != (stego_pixel[ch] & 1):
                channel_changes[ch] += 1
                pixel_changes += 1
        
        if pixel_changes > 0:
            pixels_with_changes += 1
            if pixel_changes == 1:
                pixels_with_single_change += 1
            else:
                pixels_with_multiple_changes += 1
    
    # 3. Эффективность встраивания
    efficiency = changed_bits / message_bits if changed_bits > 0 else float('inf')
    bits_per_change = message_bits / changed_bits if changed_bits > 0 else 0
    
    # 4. Визуальная метрика (насколько изменения заметны)
    visual_impact = (pixels_with_changes / len(pixels)) * 100
    
    #  Вывод результатов
    print(f"\n ОБЩАЯ СТАТИСТИКА:")
    print(f"  Всего LSB битов: {total_bits:,}")
    print(f"  Изменено битов: {changed_bits:,} ({change_percentage:.6f}%)")
    print(f"  Неизмененных битов: {unchanged_bits:,} ({100-change_percentage:.6f}%)")
    
    print(f"\n ИЗМЕНЕНИЯ ПО КАНАЛАМ:")
    channel_names = ['Красный', 'Зелёный', 'Синий']
    for ch in range(3):
        ch_percentage = (channel_changes[ch] / (len(pixels))) * 100
        print(f"  {channel_names[ch]}: {channel_changes[ch]:,} изменений ({ch_percentage:.4f}%)")
    
    print(f"\n АНАЛИЗ ПИКСЕЛЕЙ:")
    print(f"  Всего пикселей: {len(pixels):,}")
    print(f"  Пикселей с изменениями: {pixels_with_changes:,} ({visual_impact:.4f}%)")
    print(f"  Пикселей с одним изменением: {pixels_with_single_change:,} ({pixels_with_single_change/len(pixels)*100:.4f}%)")
    print(f"  Пикселей с несколькими изменениями: {pixels_with_multiple_changes:,} ({pixels_with_multiple_changes/len(pixels)*100:.4f}%)")
    
    print(f"\n⚡ ЭФФЕКТИВНОСТЬ ВСТРАИВАНИЯ:")
    print(f"  Длина сообщения: {message_bits} бит")
    print(f"  Измененных битов на 1 бит сообщения: {efficiency:.2f}")
    print(f"  Битов сообщения на 1 измененный бит: {bits_per_change:.1f}")
    
    print(f"\n ОЦЕНКА КАЧЕСТВА:")
    
    # Критерии оценки
    if change_percentage < 0.01:
        quality = " ОТЛИЧНО"
        reason = "Изменения практически незаметны (<0.01%)"
    elif change_percentage < 0.1:
        quality = " ХОРОШО"
        reason = "Минимальные изменения (0.01-0.1%)"
    elif change_percentage < 0.5:
        quality = "  УДОВЛЕТВОРИТЕЛЬНО"
        reason = "Заметные изменения (0.1-0.5%)"
    else:
        quality = " ПЛОХО"
        reason = "Слишком много изменений (>0.5%)"
    
    print(f"  Качество встраивания: {quality}")
    print(f"  Причина: {reason}")
    
    # Дополнительные метрики
    print(f"\n ДОПОЛНИТЕЛЬНЫЕ МЕТРИКИ:")
    
    # Среднее количество изменений на пиксель
    avg_changes_per_pixel = sum(channel_changes) / len(pixels)
    print(f"  Среднее изменений на пиксель: {avg_changes_per_pixel:.4f}")
    
    # Соотношение изменений к длине сообщения
    compression_ratio = message_bits / changed_bits if changed_bits > 0 else 0
    print(f"  Коэффициент сжатия (бит сообщения/изменений): {compression_ratio:.1f}:1")
    
    # Визуальное сравнение
    print(f"\n  ВИЗУАЛЬНАЯ ОЦЕНКА:")
    print(f"  Пикселей с изменениями: {pixels_with_changes:,} из {len(pixels):,}")
    print(f"  Вероятность заметить изменение: {visual_impact:.6f}%")
    
    if visual_impact < 0.01:
        print(f"    Человеческий глаз: НЕВОЗМОЖНО заметить")
    elif visual_impact < 0.1:
        print(f"    Человеческий глаз: ОЧЕНЬ СЛОЖНО заметить")
    elif visual_impact < 1:
        print(f"    Человеческий глаз: СЛОЖНО заметить")
    elif visual_impact < 5:
        print(f"    Человеческий глаз: МОЖНО заметить при детальном рассмотрении")
    else:
        print(f"    Человеческий глаз: ВОЗМОЖНО заметить")
    
    return {
        'total_bits': total_bits,
        'changed_bits': changed_bits,
        'change_percentage': change_percentage,
        'channel_changes': channel_changes,
        'pixels_with_changes': pixels_with_changes,
        'visual_impact': visual_impact,
        'efficiency': efficiency,
        'quality': quality
    }


def compare_all_stego_methods(original_path, stego_paths, descriptions, message_lengths):
    """Сравнение нескольких методов встраивания"""
    print(f"\n{'='*70}")
    print(" СРАВНИТЕЛЬНЫЙ АНАЛИЗ МЕТОДОВ СТЕГАНОГРАФИИ")
    print(f"{'='*70}")
    
    results = []
    
    for i, (stego_path, description) in enumerate(zip(stego_paths, descriptions)):
        message_bits = message_lengths[i] if i < len(message_lengths) else 18
        print(f"\n Анализ метода: {description}")
        result = compare_stego_changes(original_path, stego_path, description, message_bits)
        if result:
            results.append((description, result))
    
    # Сводная таблица
    if results:
        print(f"\n{'='*70}")
        print(" СВОДНАЯ ТАБЛИЦА РЕЗУЛЬТАТОВ")
        print(f"{'='*70}")
        
        headers = ["Метод", "Изменения %", "Качество", "Эффективность", "Пикселей с изм."]
        print(f"{headers[0]:<25} {headers[1]:<15} {headers[2]:<20} {headers[3]:<15} {headers[4]:<15}")
        print(f"{'-'*90}")
        
        for desc, res in results:
            changes_pct = f"{res['change_percentage']:.6f}%"
            quality = res['quality'].split()[1]  # Берем только слово
            efficiency = f"{res['efficiency']:.2f}"
            pixels_changed = f"{res['pixels_with_changes']:,}"
            
            print(f"{desc:<25} {changes_pct:<15} {quality:<20} {efficiency:<15} {pixels_changed:<15}")
        
        # Находим лучший метод
        best_method = min(results, key=lambda x: x[1]['change_percentage'])
        print(f"\n ЛУЧШИЙ МЕТОД: {best_method[0]}")
        print(f"   Изменений: {best_method[1]['change_percentage']:.6f}%")
        print(f"   Эффективность: {best_method[1]['efficiency']:.2f} изменений/бит")


# Пример использования
if __name__ == "__main__":
    # Сравнение отдельных методов
    print(" АНАЛИЗ КАЧЕСТВА СТЕГОГРАФИИ")
    
    # Задаем длину сообщения (18 бит сообщение + 3 бита заголовок = 21 бит)
    message_length = 21  # 18 бит данных + 3 бита заголовка
    
    result1 = compare_stego_changes(
        "test1.png", 
        "output.png", 
        "Последовательное встраивание",
        message_length
    )
    
    result2 = compare_stego_changes(
        "test1.png", 
        "encoded.png", 
        "Случайное встраивание",
        message_length
    )
    
    # Сравнение всех методов
    compare_all_stego_methods(
        original_path="test1.png",
        stego_paths=["output.png", "encoded.png"],
        descriptions=["Последовательное", "Случайное"],
        message_lengths=[21, 21]
    )

 АНАЛИЗ КАЧЕСТВА СТЕГОГРАФИИ


СРАВНЕНИЕ: Последовательное встраивание


 ОБЩАЯ СТАТИСТИКА:
  Всего LSB битов: 5,292,000
  Изменено битов: 7 (0.000132%)
  Неизмененных битов: 5,291,993 (99.999868%)

 ИЗМЕНЕНИЯ ПО КАНАЛАМ:
  Красный: 1 изменений (0.0001%)
  Зелёный: 2 изменений (0.0001%)
  Синий: 4 изменений (0.0002%)

 АНАЛИЗ ПИКСЕЛЕЙ:
  Всего пикселей: 1,764,000
  Пикселей с изменениями: 7 (0.0004%)
  Пикселей с одним изменением: 7 (0.0004%)
  Пикселей с несколькими изменениями: 0 (0.0000%)

⚡ ЭФФЕКТИВНОСТЬ ВСТРАИВАНИЯ:
  Длина сообщения: 21 бит
  Измененных битов на 1 бит сообщения: 0.33
  Битов сообщения на 1 измененный бит: 3.0

 ОЦЕНКА КАЧЕСТВА:
  Качество встраивания:  ОТЛИЧНО
  Причина: Изменения практически незаметны (<0.01%)

 ДОПОЛНИТЕЛЬНЫЕ МЕТРИКИ:
  Среднее изменений на пиксель: 0.0000
  Коэффициент сжатия (бит сообщения/изменений): 3.0:1

  ВИЗУАЛЬНАЯ ОЦЕНКА:
  Пикселей с изменениями: 7 из 1,764,000
  Вероятность заметить изменение: 0.000397%
    Человеческий глаз: НЕВОЗМО

IndexError: list index out of range

# Статистический анализ подтверждает, что метод сохраняет статистические свойства исходного контейнера. После встраивания сообщения распределение LSB-битов практически не изменилось, что делает факт встраивания статистически необнаружимым. Все тесты показывают FAIL как для оригинала, так и для стего-изображений, что доказывает корректность работы алгоритма - он не вносит дополнительных статистических аномалий